# Notebook 2: Augmented Generation: Contexto Enriquecido para Interaccionar con un LLM

Antes de construir un sistema RAG, es importante entender **cómo interactúan los modelos de lenguaje (LLMs) con los humanos** a través de lo que se llama un *prompt*.

¿Qué es un prompt?

Un *prompt* es simplemente el texto que escribimos para pedirle al modelo que genere algo. Es como darle una **instrucción o pregunta**, y el modelo responde en consecuencia.

¿Por qué es importante la fase Augmented?

Los modelos de lenguaje no lo saben todo. A veces:

- Completan lo que no saben (alucinan)
- Dan respuestas genéricas o erróneas
- Fallan si se les pregunta por apuntes personales o temas técnicos


In [2]:
import requests

OLLAMA_URL = "http://rag-ollama:11434"  
MODEL = "gemma2:2b"  

def ask_ollama(prompt: str):
    """Envía un prompt al modelo vía Ollama y devuelve la respuesta."""
    response = requests.post(
        f"{OLLAMA_URL}/api/generate",
        json={
            "model": MODEL,
            "prompt": prompt,
            "stream": False
        }
    )
    print(response.json()["response"])

ask_ollama("Hola, ¿Cómo estas?")

¡Hola! Estoy muy bien, gracias por preguntar. 😊 ¿Y tú cómo estás? 



# 1. Pregunta Objetivo

Queremos publicar un tweet con una información muy concreta:

In [12]:
question = "¿Quién está detrás de la actual estrategia económica de EE. UU.? Contesta brevemente"

ask_ollama(question)

La estrategia económica de los Estados Unidos es impulsada por una combinación de factores, incluyendo:

* **El presidente Joe Biden:** Su enfoque económico se centra en la recuperación económica post-pandemia, la inversión en infraestructura y el fortalecimiento del mercado laboral. 
* **La Casa Blanca y el Congreso:**  Estos actores establecen las políticas a nivel nacional, incluyendo la regulación fiscal, los programas sociales y el gasto público. 
* **La Federal Reserve:** El banco central de EE. UU. juega un papel crucial en la estrategia económica al controlar los tipos de interés y regular la inflación. 

En resumen, no hay una única persona "en la cabeza" de la estrategia económica. Es un esfuerzo complejo con múltiples actores y factores que se entrelazan para determinar la dirección del país.



# 2. Contexto Breve

In [13]:
context_1 = """
En enero de 2025, Kamala Harris asumió la presidencia de Estados Unidos,
impulsando una nueva estrategia económica denominada 'Green Jobs for America'.
La estrategia está dirigida principalmente por Janet Yellen,
secretaria del Tesoro, en colaboración con asesores de política energética.
"""
ask_ollama(context_1 + "\n\n" + question)

La actual estrategia económica de EE.UU., "Green Jobs for America", está impulsada por **Kamala Harris** como Presidente y  **Janet Yellen**, Secretaria del Tesoro, junto con asesores de política energética. 



# Anexo 1. Plantilla CRISPE para prompting

- Context (Contexto): será el resultado de la búsqueda en la base vectorial (información recuperada).
- Role (Rol): definimos al modelo como un experto en la materia.
- Instructions (Instrucciones): le pedimos una tarea clara y concisa.
- Steps (Pasos): indicamos cómo debe razonar o estructurar la respuesta.
- Parameters (Parámetros): restricciones que debe cumplir.
- Example (Ejemplo): añadimos un ejemplo de respuesta esperada (few-shot prompting).

## 1.1. Contexto + Rol

In [14]:
role = """
Eres un analista económico especializado en política estadounidense.
"""
ask_ollama(context_1 + "\n\n" + role + "\n\n" + question)

La actual estrategia económica de EE. UU., llamada 'Green Jobs for America', está impulsada principalmente por **Kamala Harris** como presidenta y con apoyo del **secretario del Tesoro Janet Yellen**.  Su estrategia se basa en el trabajo de asesores de política energética, quienes contribuyen a la implementación de las políticas económicas. 



## 1.2 Contexto + Rol + Instrucciones

In [15]:
inst = """
Responde de forma breve y clara, para poder publicarlo como tweet.
"""
ask_ollama(context_1 + "\n\n" + role + "\n\n" + inst + "\n\n"+ question)

La estrategia económica "Green Jobs for America" de EE.UU., impulsada por la Presidenta Kamala Harris, está liderada por la Secretaria del Tesoro Janet Yellen y asesores de política energética. 🇺🇸💰🌱  



## 1.3 Contexto + Rol + Instrucciones + Pasos

In [22]:
steps = """
1. Identifica en el contexto quién dirige la estrategia económica.
2. Responde en una sola frase.
3. Si no hay datos suficientes, responde: "No hay información en el contexto".
"""
ask_ollama(context_1 + "\n\n" + role + "\n\n" + inst + "\n\n"+ steps + "\n\n" + question)

1. La estrategia económica "Green Jobs for America" está dirigida por Janet Yellen, secretaria del Tesoro, y sus asesores de política energética. 
2. **Janet Yellen dirige la estrategia económica "Green Jobs for America"**. 



## 1.4 Contexto + Rol + Instrucciones + Pasos + Parámetros

In [23]:
params = """
- Máximo una frase.
- No inventes información.
- Usa el nombre exacto mencionado en el contexto.
"""
ask_ollama(context_1 + "\n\n" + role + "\n\n" + inst + "\n\n"+ steps + "\n\n" + params + "\n\n" + question)

La estrategia económica 'Green Jobs for America' está dirigida por Kamala Harris y Janet Yellen, secretaria del Tesoro. 



## 1.5 Contexto + Rol + Instrucciones + Pasos + Parámetros + Ejemplo

In [24]:
example = """
Pregunta: ¿Quién lideró la estrategia 'Bidenomics'?
Respuesta esperada: "Fue liderada por Joe Biden y su equipo económico."
"""
ask_ollama(context_1 + "\n\n" + role + "\n\n" + inst + "\n\n"+ steps + "\n\n" + params + "\n\n" + example + "\n\n" + question)

La estrategia 'Green Jobs for America' está dirigida por Kamala Harris y Janet Yellen. 

